In [1]:
import pandas as pd
import numpy as np

In [2]:
sizes = {'real': {'train': 443757, 'val': 214354}, 'abstract': {'train': 60000, 'val': 30000}}

In [3]:
scenario = 'real'

In [4]:
prediction = pd.read_csv('./concatenated_data/{}_prediction.csv'.format(scenario))
prediction.rename(columns={'2048':'q_type'}, inplace=True)
prediction['base_acc'] = (prediction.base_predicted == prediction.base_target).astype(int)
prediction['_set'] = ['train']*sizes[scenario]['train'] + ['val']*sizes[scenario]['val']

summary = pd.read_csv('./concatenated_data/queries_summary_{}.csv'.format(scenario))
del summary['count_top_down']
summary.rename(columns={'2048':'_q_type'}, inplace=True)

In [5]:
#fail due to randomized order
#sset_prediction = prediction[prediction._set == 'val']
#sset_prediction[['my_acc','base_acc']].sum()/sset_prediction.shape[0]

In [6]:
accurracies = prediction[['q_type','my_acc','base_acc']].groupby('q_type').sum().reset_index()

In [7]:
new_summary = pd.concat([summary, accurracies], axis=1)
assert (new_summary._q_type != new_summary.q_type).sum() == 0

In [8]:
p = 1.64

summary = new_summary.copy()
mean_dif = (summary.sum_top_down - summary.sum_bottom_up).mean()
std_dif = (summary.sum_top_down - summary.sum_bottom_up).std()
data_dif = (summary.sum_top_down - summary.sum_bottom_up)
data_diff = pd.DataFrame( (data_dif < mean_dif - p * std_dif).astype(int) + 
                         (data_dif > mean_dif + p * std_dif).astype(int) > 0 )
significantly_different = summary.loc[data_diff[data_diff[0]].index]
print(data_diff.shape)
print(np.round(float(data_dif[data_diff[0]].shape[0]) / data_diff.shape[0] * 100, 2))

(632, 1)
9.18


In [10]:
significantly_different[significantly_different._count>2].sort_values(by='winner')

,_q_type,sum_top_down,sum_bottom_up,count_bottom_up,diff,_count,over_median,winner,q_type,my_acc,base_acc
59,can elephants,0.464519,0.535481,0.5,-0.07,5,True,bottom_up,can elephants,5,5
412,when were,0.449654,0.550346,0.5,-0.10,10,True,bottom_up,when were,6,6
570,will those,0.467417,0.532583,0.5,-0.07,7,True,bottom_up,will those,4,5
259,natural or,0.461028,0.538972,0.5,-0.08,22,True,bottom_up,natural or,20,20
258,natural habitat,0.462729,0.537271,0.5,-0.07,203,True,bottom_up,natural habitat,184,186
315,small or,0.466027,0.533973,0.5,-0.07,20,True,bottom_up,small or,15,16
223,it's the,0.466928,0.533072,0.5,-0.07,5,True,bottom_up,it's the,3,4
186,houses or,0.428469,0.571531,0.5,-0.14,3,False,bottom_up,houses or,3,3
183,hazy or,0.466019,0.533981,0.5,-0.07,21,True,bottom_up,hazy or,17,20
94,cloudy or,0.468010,0.531990,0.5,-0.06,4,False,bottom_up,cloudy or,4,4
